# 训练

In [ ]:
import pandas as pd

df = pd.read_csv('train_cleaned.csv')

将数据分为feature和label

In [ ]:
label_df = df['label']
feature_df = df.drop(['id', 'label'], axis=1)

## 分割数据

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature_df, label_df, test_size=0.3)

## 尝试不同算法

### 逻辑回归
macro_F1 = 0.20
y_test是测试集的真实标签，model.predict(X_test)是模型对测试集的预测标签。

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(multi_class='ovr', solver='liblinear')
model = lr.fit(X_train, np.ravel(y_train))

## 计算 macro F1

In [ ]:
from sklearn.metrics import f1_score

macro_F1 = f1_score(y_test, model.predict(X_test), average='macro')
print("macro_F1 is {}".format(macro_F1))

输出分类的报告得到关于模型的更多的细节

In [21]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))